In [ ]:
!pip install qiskit qiskit-ibm-runtime pylatexenc qiskit_aer

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
import numpy as np
# 임의의 s 에 대한 n-qubit 의 번스타인-바지라니 오라클 회로를 구현하는 함수
def bv_oracle_circuit(n):
    qy = QuantumRegister(1, 'y')
    qx = QuantumRegister(n, 'x')
    qc = QuantumCircuit(qy, qx) # qc 라는 이름의 n+1큐비트 양자 회로 객체를 생성

    np.random.seed()       # numpy 의 random  함수를 초기화
    s = np.random.randint(2,size=n) # [0,2) 의 정수를 무작위로 n 개 생성
    print ("s = ", s)
    # implement |x,y> ->|x, y XOR f(X)>
    # where f(x) = x[0]s[0] XOR x[1]s[1] ... XOR x[n-1]s[n-1]
    for i in range(n):
        if s[i] == 1:      # 생성된 i 번째 난수가 1인 경우에만
            qc.cx(qx[i],qy)   # 제어 큐비트 n, 타겟 큐비트 0 인 CX(CNOT) gate
    qc.name = "BV Oracle"
    return qc              # qc 객체를 반환

circuit = bv_oracle_circuit(5)
circuit.draw(output='mpl')

In [ ]:
# 번스타인-바지라니 오라클 함수를 호출하여 번스타인-바지라니 회로를 circuit 이라는 이름의 객체로 구현
n = 5
qx = QuantumRegister(n,'x')  # n 큐비트의 양자 레지스터 객체를 qx 라는 이름으로 생성, 회로에는 'x'로 표시
qy = QuantumRegister(1,'y')  # 1 큐비트의 양자 레지스터 객체를 qy 라는 이름으로 생성, 회로에는 'y'로 표시
c = ClassicalRegister(n,'c') # 1 비트의 고전 레지스터 객체를 c 라는 이름으로 생성, 회로에는 'c'로 표시
circuit = QuantumCircuit(qy, qx, c) # qy,qx,c 를 가지는 양자 회로를 circuit 이라는 이름의 객체로 생성
circuit.x(qy)                # qy 큐비트에 X gate -> qy 큐비트 상태가 |1> 이 됨
circuit.h(qy)                # qy 큐비트에 H gate -> qy 큐비트 상태가 |-> 이 됨
circuit.h(qx)                # qx 의 모든 큐비트에 H gate 적용
circuit.barrier()
# bv_oracle_circuit() 을 호출하여 오라클 회로를 circuit 에 추가
circuit.compose(bv_oracle_circuit(n), range(n+1), inplace=True)
circuit.barrier()
circuit.h(qx)              # qx 의 모든 큐비트에 H gate 적용
circuit.measure(qx,reversed(c)) # qx 레지스터의 모든 큐비트들을 c 레지스터에서 역순으로 측정하는 회로를 구현
circuit.draw(output='mpl') # circuit 회로를 matplot library 형식의 그림 화일로 출력

In [ ]:
# AerSimulator 를 이용하여 circuit 회로를 측정한 결과를 출력
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_aer import AerSimulator

aer_sim = AerSimulator()
pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=1)

isa_circuit = pm.run(circuit)
sampler = Sampler(mode=aer_sim)
job = sampler.run([isa_circuit], shots=1)
result = job.result()
count = result[0].data.c.get_counts()
print (count)

In [ ]:
print (f'The hidden string s is {list(count.keys())[0]}')
